In [1]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
import time

In [2]:
cg = CoinGeckoAPI()

## 1. `cg.get_coin_market_chart_by_id()` API


- **:param `id`:**          default is Bitcoin "bitcoin"
- **:param `vs_currency`:** default is USD "usd"
- **:param `days`:**        the range of days to retireve data for "1", "30", "max", etc.
- **:param `interval`:**    the data interval "minutely", "hourly", "daily"

In [63]:
data = cg.get_coin_market_chart_by_id(id = "bitcoin",
                                      vs_currency = "usd",
                                      days = "5",
                                      interval = "daily")

###
# :param id:          default is Bitcoin "bitcoin"
# :param vs_currency: default is USD "usd"
# :param days:        the range of days to retireve data for "1", "30", "max", etc.
# :param interval:    the data interval "minutely", "hourly", "daily"
###

In [64]:
type(data)

dict

In [65]:
#data: a dictionary with 3 keys: prices, market_caps and total_volumes

#each key associates with a list, in which each element is also a list of two elements: timestamp and value of the key

data

{'prices': [[1729468800000, 68962.82918045693],
  [1729555200000, 67394.85775735618],
  [1729641600000, 67351.04793512498],
  [1729728000000, 66683.96806542306],
  [1729814400000, 68214.05205157402],
  [1729874784000, 67749.91021477139]],
 'market_caps': [[1729468800000, 1362381684459.9053],
  [1729555200000, 1332326858574.4448],
  [1729641600000, 1331426858508.4004],
  [1729728000000, 1318285437546.2786],
  [1729814400000, 1347888109010.5889],
  [1729874784000, 1337327368146.5579]],
 'total_volumes': [[1729468800000, 17253863432.786995],
  [1729555200000, 40485496091.95008],
  [1729641600000, 31398389967.38113],
  [1729728000000, 32676918488.059],
  [1729814400000, 35892606570.03549],
  [1729874784000, 36796488933.79797]]}

In [66]:
data["prices"]

[[1729468800000, 68962.82918045693],
 [1729555200000, 67394.85775735618],
 [1729641600000, 67351.04793512498],
 [1729728000000, 66683.96806542306],
 [1729814400000, 68214.05205157402],
 [1729874784000, 67749.91021477139]]

In [67]:
df = pd.DataFrame(data["prices"], columns = ["Timestamp", "Price"])
df.head()

,Timestamp,Price
0,1729468800000,68962.829180
1,1729555200000,67394.857757
2,1729641600000,67351.047935
3,1729728000000,66683.968065
4,1729814400000,68214.052052


### Convert Unix timestamp (in milliseconds) into standard date-time format

**`pd.to_datetime()`**

In [68]:
df["Date"] = pd.to_datetime(df["Timestamp"], unit="ms")

In [69]:
df.head()

,Timestamp,Price,Date
0,1729468800000,68962.829180,2024-10-21
1,1729555200000,67394.857757,2024-10-22
2,1729641600000,67351.047935,2024-10-23
3,1729728000000,66683.968065,2024-10-24
4,1729814400000,68214.052052,2024-10-25


### Set the "Date” (date-time values) as the DataFrame's index

Many Pandas time-based operations (like `resample()`) work based on this index

In [70]:
df.set_index("Date", inplace = True)  # inplace=Ture: modifies the original df directly rather than returning a new df
df.head()

,Timestamp,Price
Date,,
2024-10-21,1729468800000,68962.829180
2024-10-22,1729555200000,67394.857757
2024-10-23,1729641600000,67351.047935
2024-10-24,1729728000000,66683.968065
2024-10-25,1729814400000,68214.052052


## 2. `cg.get_coin_market_chart_range_by_id()` API

- **:param `id`:** default is Bitcoin "bitcoin"
- **:param `vs_currency`:** default is USD "usd"
- **:param `from_timestamp`:** the Unix timestamp
- **:param `to_timestamp`:** the Unix timestamp

In [72]:
start_date = "2024-01-01"
end_date = "2024-02-01"

### Convert start and end dates to Unix timestamps

In [45]:
pd.to_datetime(start_date)

Timestamp('2024-01-01 00:00:00')

In [46]:
pd.to_datetime(start_date).timetuple()

time.struct_time(tm_year=2024, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=0, tm_yday=1, tm_isdst=-1)

In [47]:
time.mktime(pd.to_datetime(start_date).timetuple())

1704085200.0

In [48]:
int(time.mktime(pd.to_datetime(start_date).timetuple()))

1704085200

In [73]:
from_timestamp = int(time.mktime(pd.to_datetime(start_date).timetuple()))
to_timestamp = int(time.mktime(pd.to_datetime(end_date).timetuple()))

### Get the data

In [74]:
data = cg.get_coin_market_chart_range_by_id(id = "bitcoin",
                                            vs_currency = "usd",
                                            from_timestamp = from_timestamp,
                                            to_timestamp = to_timestamp)
type(data)

dict

In [75]:
df = pd.DataFrame(data["prices"], columns = ["Timestamp", "Price"])

df.head()                  

,Timestamp,Price
0,1704085264933,42349.340091
1,1704088850683,42245.399145
2,1704092451846,42417.935946
3,1704096005236,42471.252137
4,1704099653385,42517.226836


In [76]:
df["Date"] = pd.to_datetime(df["Timestamp"], unit = "ms")
df.tail()

,Timestamp,Price,Date
738,1706745650896,42572.119544,2024-02-01 00:00:50.896
739,1706749265588,42451.351597,2024-02-01 01:01:05.588
740,1706752848843,41971.370820,2024-02-01 02:00:48.843
741,1706756468307,42102.189965,2024-02-01 03:01:08.307
742,1706760069226,41972.164605,2024-02-01 04:01:09.226


In [77]:
df.set_index("Date", inplace=True)
df.head()

,Timestamp,Price
Date,,
2024-01-01 05:01:04.933,1704085264933,42349.340091
2024-01-01 06:00:50.683,1704088850683,42245.399145
2024-01-01 07:00:51.846,1704092451846,42417.935946
2024-01-01 08:00:05.236,1704096005236,42471.252137
2024-01-01 09:00:53.385,1704099653385,42517.226836


### Using `resample()` to get 4-hourly data

In [78]:
df_4h = df.resample('4h').first()

df_4h.head()

,Timestamp,Price
Date,,
2024-01-01 04:00:00,1704085264933,42349.340091
2024-01-01 08:00:00,1704096005236,42471.252137
2024-01-01 12:00:00,1704110486723,42698.693073
2024-01-01 16:00:00,1704124845648,42799.529566
2024-01-01 20:00:00,1704139213477,43495.913496


## 3. CoinGecko Data Generator

In [ ]:
import pandas as pd
from pycoingecko import CoinGeckoAPI
import time


def generate_data_coingecko(outpath, coin_id = "bitcoin", vs_currency = "usd",
                            start_date = "2024-01-01", end_date = "2024-07-01",
                            intervel = "4h", use_range_api = True):
    """
    Fetch cryptocurrency data using either the range API or the chart API from CoinGeckoAPI,
    resample to the desired interval, and save it as a CSV file.

    :param output_path: Path to save the CSV file.
    :param coin_id: CoinGecko ID of the cryptocurrency (default is Bitcoin "bitcoin").
    :param vs_currency: The currency in which to get the market data (default is USD "usd").
    :param start_date: The start date for the data (format 'YYYY-MM-DD').
    :param end_date: The end date for the data (format 'YYYY-MM-DD').
    :param interval: Desired resampling interval (e.g., '4h' for 4-hour intervals).
    :param use_range_api: Boolean to determine whether to use the range API (True) or the chart API (False).
    """
    
    # Initialize CoinGecko API
    cg = CoinGeckoAPI()

    if use_range_api:
        #Convert start and end dates to Unix timestamp
        from_timestamp = int(time.mktime(pd.to_datetime(start_date).timetuple()))
        to_timestamp = int(time.mktime(pd.to_datetime(end_date).timetuple()))

        data = cg.get_coin_market_chart_range_by_id(id = coin_id, vs_currency = vs_currency,
                                                    from_timestamp = from_timestamp, to_timestamp = to_timestamp)

    else:
        days = 
    










    

In [80]:
import requests
import pandas as pd

In [90]:
url = "https://api.coincap.io/v2/assets/ethereum/history?interval=d1"
response = requests.get(url)
data = response.json()


In [91]:
data

{'data': [{'priceUsd': '1784.6201554702123418',
   'time': 1698364800000,
   'date': '2023-10-27T00:00:00.000Z'},
  {'priceUsd': '1788.0622255959847182',
   'time': 1698451200000,
   'date': '2023-10-28T00:00:00.000Z'},
  {'priceUsd': '1791.8593855739585031',
   'time': 1698537600000,
   'date': '2023-10-29T00:00:00.000Z'},
  {'priceUsd': '1804.0195782396389757',
   'time': 1698624000000,
   'date': '2023-10-30T00:00:00.000Z'},
  {'priceUsd': '1804.9241300449436976',
   'time': 1698710400000,
   'date': '2023-10-31T00:00:00.000Z'},
  {'priceUsd': '1814.1532215049560994',
   'time': 1698796800000,
   'date': '2023-11-01T00:00:00.000Z'},
  {'priceUsd': '1828.7077244158950642',
   'time': 1698883200000,
   'date': '2023-11-02T00:00:00.000Z'},
  {'priceUsd': '1805.7247549687920210',
   'time': 1698969600000,
   'date': '2023-11-03T00:00:00.000Z'},
  {'priceUsd': '1840.2426655254526457',
   'time': 1699056000000,
   'date': '2023-11-04T00:00:00.000Z'},
  {'priceUsd': '1884.2120219367926039'

In [92]:
df = pd.DataFrame(data['data'])
df['date'] = pd.to_datetime(df['time'], unit='ms')
df.set_index('date', inplace=True)
    
    # 只选择相关的列
df = df[['priceUsd']].rename(columns={'priceUsd': 'Price'})

In [93]:
df.head()

,Price
date,
2023-10-27,1784.6201554702123418
2023-10-28,1788.0622255959847182
2023-10-29,1791.8593855739585031
2023-10-30,1804.0195782396389757
2023-10-31,1804.9241300449436976


In [94]:
df.tail()

,Price
date,
2024-10-20,2672.1423593761332751
2024-10-21,2702.4709225433300124
2024-10-22,2633.9876711020570245
2024-10-23,2565.1936050129203745
2024-10-24,2534.2717917868754643


In [95]:
import yfinance as yf
import pandas as pd

# 提取以太坊（ETH-USD）的日线数据，时间范围从2015年1月1日到今天
eth_daily_data = yf.download('ETH-USD', start='2015-01-01', end='2024-10-01', interval='1d')


ModuleNotFoundError: No module named 'yfinance'